In [1]:
X.groupby("BatchId")[["Amount", "Value", "Day", "Hour", "week_day", "weeks"]].agg(
    ["count", "min", "max", "mean"]
)

df.groupby("A").agg({"B": ["min", "max"], "C": "sum"})

NameError: name 'X' is not defined

In [ ]:
train_y1 = train_y.copy()
train_X1 = train_X.copy()
val_y1 = val_y.copy()
val_X1 = val_X.copy()
train_X1.drop(cols_cat, axis=1, inplace=True)  # Delete categorical data
val_X1.drop(cols_cat, axis=1, inplace=True)  # Delete categorical data
# train_X1 = train_X.select_dtypes(exclude=['object'])
# val_X1 = val_X.select_dtypes(exclude=['object'])
train_X1.head()

In [ ]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes, random_state=1)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return mae


for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X1, val_X1, train_y1, val_y1)
    print(
        "Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" % (max_leaf_nodes, my_mae)
    )

In [ ]:
model_1 = RandomForestClassifier(n_estimators=50, random_state=0)
model_2 = RandomForestClassifier(n_estimators=100, random_state=0)
model_3 = RandomForestClassifier(n_estimators=100, random_state=0)
model_4 = RandomForestClassifier(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestClassifier(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]


# Function for comparing different models
def score_model(model, X_t=train_X1, X_v=val_X1, y_t=train_y1, y_v=val_y1):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)


for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i + 1, mae))

In [ ]:
first_model_decision_tree = DecisionTreeClassifier(random_state=1)
first_model_decision_tree.fit(train_X, train_y)
first_model_decision_tree_predicted = first_model_decision_tree.predict(val_X)

df_with_res = val_X.copy()
df_with_res = df_with_res.join(val_y.copy())
df_with_res["Predictions"] = list(first_model_decision_tree_predicted)
print(
    f'Nombre de prédictions correcte : {df_with_res.Predictions[(df_with_res["Predictions"] == df_with_res["FraudResult"] )].count()}'
)
print(
    f'Nombre de prédictions incorrecte : {df_with_res.Predictions[(df_with_res["Predictions"] != df_with_res["FraudResult"] )].count()}'
)
print(
    f'Nombre de fraude non prédites : {df_with_res.Predictions[(df_with_res["Predictions"] != df_with_res["FraudResult"] )&(df_with_res["FraudResult"]==1)].count()}'
)
print(
    f'Nombre de pas fraude prédite en fraude : {df_with_res.Predictions[(df_with_res["Predictions"] != df_with_res["FraudResult"] )&(df_with_res["FraudResult"]==0)].count()}'
)
print(
    f'Nombre de fraude bien prédite : {df_with_res.Predictions[(df_with_res["Predictions"] == df_with_res["FraudResult"] )&(df_with_res["FraudResult"]==1)].count()}'
)
print(
    f'Nombre de pas fraude bien prédite : {df_with_res.Predictions[(df_with_res["Predictions"] == df_with_res["FraudResult"] )&(df_with_res["FraudResult"]==0)].count()}'
)
print(f"Nombre de Fraude dans le dataset : {val_y.value_counts()[1]}")
print(f"Nombre total : {df_with_res.Predictions.count()}")
print(
    df_with_res[["FraudResult", "Predictions"]][
        (df_with_res["Predictions"] != df_with_res["FraudResult"])
    ]
)
df_with_res.head(10)
report(
    val_y,
    first_model_decision_tree_predicted,
    "Decision Tree Classifier",
    "first simple model",
    csvw=False,
)